# Procedural Forest Visualisation - Isaac Gym

## Contents

1. Initialise Isaac Gym
2. Load & display domain randomised lsystem trees.
3. If alignment rotation file exists, use that to oritent the trees.

## Conf

In [1]:
spacing = 2.0
use_gpu_pipeline = True
enable_viewer = True

## Support Functions

### Context & Logging

In [2]:
import os
import sys
import logging

spot_path = os.getenv('spot_path')
source_dir = f"{spot_path}/source/"
notebooks_dir = f"{spot_path}/notebooks/work2/"

if source_dir not in sys.path:
    sys.path.append(source_dir)

### Simulation

In [3]:
import math
import numpy as np
import time
import random
import pickle

from isaacgym import gymapi
from isaacgym import gymutil
from isaacgym import gymtorch

import torch
import matplotlib
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import BoxStyle

from collections import namedtuple as nc
from estimation.isaac.stepper import Stepper
from common.helpers import futils
from reinforcement.ige.helpers import pos_locator
from reinforcement.ige.helpers import physics_helper, rl_helper

Importing module 'gym_38' (/home/jay/Desktop/jay/grad/usyd/research/roger/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/jay/Desktop/jay/grad/usyd/research/roger/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.0.1+cu117
Device count 1
/home/jay/Desktop/jay/grad/usyd/research/roger/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/jay/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/jay/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


In [4]:
torch.set_printoptions(threshold=8)
torch.manual_seed(970)
np.random.seed(970)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu_pipeline else 'cpu')
logging.info(f"Using device {device} for the pipeline")

In [6]:
# Initialize gym
gym = gymapi.acquire_gym()

# Parse arguments
args = gymutil.parse_arguments(
    description="Tree Loader")

logging.info(f"args:{args}")

In [7]:
# configure sim
sim_params = gymapi.SimParams()
sim_params.dt = 1.0 / 60.0
sim_params.substeps = 2

sim_params.up_axis = gymapi.UP_AXIS_Z
sim_params.gravity = gymapi.Vec3(0.0, 0.0, -9.8)


if args.physics_engine == gymapi.SIM_FLEX:
    sim_params.flex.solver_type = 5
    sim_params.flex.num_outer_iterations = 4
    sim_params.flex.num_inner_iterations = 15
    sim_params.flex.relaxation = 0.75
    sim_params.flex.warm_start = 0.8
elif args.physics_engine == gymapi.SIM_PHYSX:
    sim_params.physx.solver_type = 1
    sim_params.physx.num_position_iterations = 4
    sim_params.physx.num_velocity_iterations = 1
    sim_params.physx.num_threads = args.num_threads
    sim_params.physx.use_gpu = args.use_gpu

sim_params.use_gpu_pipeline = use_gpu_pipeline

if not args.use_gpu_pipeline:
    logging.warn("WARNING: Forcing CPU pipeline.")

sim = gym.create_sim(args.compute_device_id,
                     args.graphics_device_id, args.physics_engine, sim_params)

if sim is None:
    logging.error("*** Failed to create sim")
    quit()

# Create viewer
viewer = gym.create_viewer(sim, gymapi.CameraProperties()) if enable_viewer else None
if enable_viewer and viewer is None:
    logging.error("*** Failed to create viewer")
    quit()


# Add ground plane
plane_params = gymapi.PlaneParams()

plane_params.normal = gymapi.Vec3(0, 0, 1)  # z-up!
plane_params.distance = 0

# create the ground plane
gym.add_ground(sim, plane_params)

## Load Trees

In [8]:
ternary_class = 'ta'
asset_root = spot_path+f"/source/simulation/lsystems2/three/urdf/tree/gen/pliable07/{ternary_class}/test/"
tree_alignment_quats_file = os.path.join(asset_root, 'tree_alignment_quats.pkl')

In [9]:
num_tree_types = futils.count_sub_folders(asset_root)
num_envs = num_tree_types

In [10]:
gt_tree_asset_files = [f"{ternary_class}_{r_idx}/ternary_{ternary_class[-1]}.urdf" for r_idx in range(num_tree_types)]

tree_alignment_quats = None
if os.path.exists(tree_alignment_quats_file):
    with open(tree_alignment_quats_file, 'rb') as file:
        tree_alignment_quats = pickle.load(file)

    assert len(tree_alignment_quats) == num_tree_types, "invalid alignment information, only one test can be removed..."

In [11]:
gt_tree_asset_options = gymapi.AssetOptions()
gt_tree_asset_options.fix_base_link = True
gt_tree_asset_options.armature = 0.01
gt_tree_asset_options.collapse_fixed_joints=True
    
gt_tree_asset_options.override_com = True
gt_tree_asset_options.override_inertia = True

gt_tree_assets = []

for i in range(num_tree_types):
    _gt_tree_asset = gym.load_asset(sim, asset_root, gt_tree_asset_files[i], gt_tree_asset_options)
    gt_tree_assets.append(_gt_tree_asset)

In [12]:
# set up the env grid
env_lower = gymapi.Vec3(-spacing, -spacing,  0.0)
env_upper = gymapi.Vec3(spacing, spacing, spacing)
num_per_row = int(math.sqrt(num_envs))

# cache some common handles for later use
envs = []
gt_actor_handles = []

sim_actor_handles = []

# create and populate the environments
for env_i in range(num_envs):
    env = gym.create_env(sim, env_lower, env_upper, num_per_row)
    envs.append(env)

    gt_tree_pose = gymapi.Transform()
    gt_tree_pose.p = gymapi.Vec3(0, 0, 0.0)
    if tree_alignment_quats is not None:
        gt_tree_pose.r = tree_alignment_quats[env_i]

    gt_actor_handle = gym.create_actor(
        env, gt_tree_assets[env_i], gt_tree_pose, f"GroundTruthTree_{env_i}", env_i, 1)
    
    gt_actor_handles.append(gt_actor_handle)

In [13]:
if len({*gt_actor_handles}) != 1:
    raise ValueError("Invalid Actor Handles Created")

In [14]:
if enable_viewer:
    cam_pos = gymapi.Vec3(0, 0, 4)
    cam_target = gymapi.Vec3(5, 5, 1)
    num_per_row = int(num_envs ** 0.5)
    middle_env = envs[num_envs // 2 + num_per_row // 2]

    gym.viewer_camera_look_at(viewer, envs[0], cam_pos, cam_target)

## Set Up Environment

### Get State Tensors

In [15]:
gym.prepare_sim(sim)

True

In [16]:
_root_tensor = gym.acquire_actor_root_state_tensor(sim)
root_tensor = gymtorch.wrap_tensor(_root_tensor)
print(root_tensor.shape)

torch.Size([512, 13])


In [17]:
_dof_states = gym.acquire_dof_state_tensor(sim)
dof_states = gymtorch.wrap_tensor(_dof_states)

In [18]:
_rb_states = gym.acquire_rigid_body_state_tensor(sim)
rb_states = gymtorch.wrap_tensor(_rb_states)

In [19]:
stepper = Stepper(gym, sim, viewer)

### Add Dynamics

In [20]:
 for env_i in range(num_envs):
    tree_dof_props = gym.get_asset_dof_properties(gt_tree_assets[env_i])
    num_tree_dofs = gym.get_asset_dof_count(gt_tree_assets[env_i])

    for dof_i in range(num_tree_dofs):
        noise_std = 1.
        # use rudimentary policy for test and train
        branch_level = rl_helper.branch_level(gym.get_asset_dof_name(gt_tree_assets[env_i], dof_i))
        kp, kd = physics_helper.rud_deflection_param(branch_level=branch_level, base_kp=400,
                                                             noise_std=noise_std)
        tree_dof_props['stiffness'][dof_i] = kp
        tree_dof_props['damping'][dof_i] = kd
        tree_dof_props['friction'][dof_i] = 0.01
        tree_dof_props['effort'][dof_i] = 100  # a high value for the tree to stand up straight.

        gym.set_actor_dof_properties(envs[env_i], gt_actor_handles[env_i], tree_dof_props)

## Simulate & View

In [ ]:
for step_idx in range(2000):
    gym.refresh_actor_root_state_tensor(sim)
    gym.refresh_dof_state_tensor(sim)
    gym.refresh_rigid_body_state_tensor(sim)
    stepper.step_through(steps=1)